# NDVI Calculation

In [1]:
import sys
sys.path.append('../')

In [2]:
import datetime as dt
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import lithops
import time
import shutil
import pylab
import os
import gc
import mgrs
from rasterio.io import MemoryFile
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
from lithops import Storage

import cloudbutton_geospatial.s2froms3 as s2froms3
from cloudbutton_geospatial.utils import notebook as notebook_utils
from cloudbutton_geospatial.io_utils.ndvi import get_ndvi_params, ndvi_calculation, ndvi_tile_sentinel, get_subset_raster, lonlat_to_utm, get_poly_within
from cloudbutton_geospatial.io_utils.plot import tiff_overview, plot_map

%matplotlib inline

## Input parameters

Select the date interval in which tiles will be processed:

In [3]:
BUCKET_NAME = "daniel-lithops-geospatial"

In [4]:
from_date, to_date = notebook_utils.pick_date_range()

DatePicker(value=datetime.date(2019, 9, 17), description='From day')

DatePicker(value=datetime.date(2021, 9, 16), description='To day')

Select the tile's cloud percentage threshold:

In [5]:
percentage = notebook_utils.pick_percentage_slider()

IntSlider(value=0, continuous_update=False, description='Percentage of cloudiness')

In [ ]:
percentage = notebook_utils.pick_percentage_slider()

Select the area which delimites the tiles you want to process (left click to mark a point in the map, right click to erase current selection):

In [ ]:
map_region = notebook_utils.MapRegion()

In [ ]:
coords = []
lats = []
lons = []

for value in map_region.get_region()[:-1]:
    coords.append(value)
    lats.append(value[1])
    lons.append(value[0])

start_date = from_date.value  # Start date to search images
end_date = to_date.value  # End date to search images
what = ['B04', 'B08']  # What we want to download
cc = percentage.value  # Minimum cloud cover on each image, 25 is 25%

#for lon, lat in zip(lons, lats):
    #print([lon, lat], start_date, end_date, what, cc)
    
lons = [-123.56323, -121.25610, -123.56323, -121.27258]    
lats = [39.28755, 39.28755, 37.48794, 37.49665]
points = []
for lon, lat in zip(lons, lats):
    points.append([lon, lat])
    print([lon, lat], start_date, end_date, what, cc)

print(points)
#map_region.get_region()[:-1]

In [ ]:
import math


def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [ ]:
# Centro del poligono
centroid = (sum(lats) / len(lats), sum(lons) / len(lons))
centroid

In [ ]:
i, p = 0, 0


while i != len(points):
    p = i + 1
    
    while p != len(points):
        
        dis = distance(points[i], points[p])
        divisions = int(dis / 100)
        
        if divisions > 0:
            toSum = [(points[i][0] - points[p][0]) / (divisions + 1) , (points[i][1] - points[p][1]) / (divisions + 1)]
            
            while divisions != 0:
                lons.append(points[i][0] - (toSum[0] * divisions))
                lats.append(points[i][1] - (toSum[1] * divisions))
                divisions = divisions - 1
        p = p + 1 
    i = i + 1
    

In [ ]:
mgrsObject = mgrs.MGRS()
c = mgrsObject.toMGRS("40.65479964952847", "-124.18269962265676", MGRSPrecision=0)
#print(c)

a = mgrsObject.toLatLon("10TDL")
b = mgrsObject.toMGRS(a[0], a[1])
#print( mgrsObject.toLatLon("10TDL"))

for lon, lat in zip(lons, lats):
    print(mgrsObject.toMGRS(lat, lon, MGRSPrecision=0))
    
    
print(mgrsObject.toLatLon("10TDL"))
print(mgrsObject.toLatLon("10TFL"))
print(mgrsObject.toLatLon("10SFJ"))
print(mgrsObject.toLatLon("10SDJ"))

print(ord("F"))
print(chr(70))

In [ ]:
mgrsObject = mgrs.MGRS()
for longitude, latency in zip(lons, lats):
    zone = mgrsObject.toMGRS(latency, longitude, MGRSPrecision=0)
    

In [33]:
mgrsObject = mgrs.MGRS()
for longitude, latency in zip(lons, lats):
    zone = mgrsObject.toMGRS(latency, longitude, MGRSPrecision=0)
    

## Get Sentinel-2 packages from a indefinite number of cells

In [103]:
scenes_f1 = []
scenes_f2 = []

for longitude, latency in zip(lons, lats):
    try:
        # Get scenes from intital date
        f1 = s2froms3.get_scene_list(lon=longitude, lat=latency, start_date=start_date, end_date=start_date,
        what=what, cloud_cover_le=cc)

        # Get scenes from end date
        f2 = s2froms3.get_scene_list(lon=longitude, lat=latency, start_date=end_date, end_date=end_date,
        what=what, cloud_cover_le=cc)

        if len(scenes_f1) == 0:
            scenes_f1.append(f1)
            scenes_f2.append(f2)

        if f1 not in scenes_f1:
            scenes_f1.append(f1)
            scenes_f2.append(f2)

        print(f'Found scenes {start_date}:', f1)
        print(f'Found scenes {end_date}:', f2)
        print(f'Lon: {longitude}, Lat: {latency}')
        print(f'Cell: {f1[0][0].split("/")[2]} {f1[0][0].split("/")[3]} {f1[0][0].split("/")[4]}\n')
    
    except Exception:
        pass


if len(scenes_f1) == 0:
    raise Exception('No data found')

scene = scenes_f1[-1][-1]
scene_band = rasterio.open('s3://'+scene[0])
windows = list(scene_band.block_windows())

Found scenes 2019-09-17: [('sentinel-cogs/sentinel-s2-l2a-cogs/10/S/DJ/2019/9/S2A_10SDJ_20190917_0_L2A/B04.tif', 'sentinel-cogs/sentinel-s2-l2a-cogs/10/S/DJ/2019/9/S2A_10SDJ_20190917_0_L2A/B08.tif')]
Found scenes 2021-09-16: [('sentinel-cogs/sentinel-s2-l2a-cogs/10/S/DJ/2021/9/S2A_10SDJ_20210916_0_L2A/B04.tif', 'sentinel-cogs/sentinel-s2-l2a-cogs/10/S/DJ/2021/9/S2A_10SDJ_20210916_0_L2A/B08.tif')]
Lon: -123.56323, Lat: 39.28755
Cell: 10 S DJ

Found scenes 2019-09-17: [('sentinel-cogs/sentinel-s2-l2a-cogs/10/S/FJ/2019/9/S2A_10SFJ_20190917_0_L2A/B04.tif', 'sentinel-cogs/sentinel-s2-l2a-cogs/10/S/FJ/2019/9/S2A_10SFJ_20190917_0_L2A/B08.tif')]
Found scenes 2021-09-16: [('sentinel-cogs/sentinel-s2-l2a-cogs/10/S/FJ/2021/9/S2A_10SFJ_20210916_0_L2A/B04.tif', 'sentinel-cogs/sentinel-s2-l2a-cogs/10/S/FJ/2021/9/S2A_10SFJ_20210916_0_L2A/B08.tif')]
Lon: -121.2561, Lat: 39.28755
Cell: 10 S FJ

Found scenes 2019-09-17: [('sentinel-cogs/sentinel-s2-l2a-cogs/10/S/EJ/2019/9/S2A_10SEJ_20190917_0_L2A/B04.ti

In [106]:
storage = lithops.storage.Storage()

2022-06-10 13:09:14,838 [DEBUG] lithops.config -- Loading configuration from C:\Users\alega\PycharmProjects\geospatial-usecase\ndvi-diff\.lithops_config
2022-06-10 13:09:14,845 [DEBUG] lithops.config -- Loading Storage backend module: aws_s3
2022-06-10 13:09:14,845 [DEBUG] lithops.storage.backends.aws_s3.aws_s3 -- Creating S3 client
2022-06-10 13:09:14,851 [INFO] lithops.storage.backends.aws_s3.aws_s3 -- S3 client created - Region: us-east-1


## Parallel execution in the cloud

In [147]:
def calculate_ndvi(scene, ij_window, storage):

    ij, window = ij_window
    band_4_s3_loc, band_8_s3_loc = scene
    band_path = band_4_s3_loc.split('/')
    ndvi_local = f'/tmp/{band_path[7]}_{ij}_NDVI.tif'
    jpg_local = f'/tmp/{band_path[7]}_{ij}_NDVI.jpg'

    # generate nir and red objects as arrays in float64 format
    # Read from S3 to lambda
    
    get_t0 = time.time()
    band4 = rasterio.open('s3://'+band_4_s3_loc)  # red
    band8 = rasterio.open('s3://'+band_8_s3_loc)  # nir
    
    
    profile = band4.profile
    profile.update(dtype='float64')
    profile.update(width=window.width)
    profile.update(height=window.height)

    with rasterio.open(ndvi_local, 'w', **profile) as dst:
        red = band4.read(1, window=window).astype('float64')
        nir = band8.read(1, window=window).astype('float64')
        get_t1 = time.time()
        ndvi = (np.where((nir + red) == 0., 0, (nir - red) / (nir + red))).astype('float64')
        ndvi_mean = np.mean(ndvi, axis=0)
        dst.write(ndvi, 1)
        ndvi[0][0] = -1
        ndvi[0][1] = 1
        plt.imsave(jpg_local, ndvi, cmap="RdYlGn")
    
    # THROUGHPUT WRITE    
    with open(jpg_local, 'rb') as jpg_temp:
        put_t0 = time.time()
        co_jpg = storage.put_cloudobject(jpg_temp.read(), key=jpg_local.replace('/tmp/', ''))
        put_t1 = time.time()
        
        obj_metadata = storage.head_object(BUCKET_NAME, jpg_local.replace('/tmp/', ''))
        put_sz = float(obj_metadata.get('content-length'))
    
    write_bandwidth_mb = put_sz / (put_t1 - put_t0) / 1e6
          
    stats = {
        'put': {'t0': put_t0, 't1': put_t1, 'bandwidth': write_bandwidth_mb, 'size': put_sz},
        'get': {'t0': get_t0, 't1': get_t1, 'bandwidth': 0, 'size': 0},
    }
    
    return ndvi_local, ndvi_mean, co_jpg, stats 


def compute_ndvi_diff(old_scene, new_scene, ij_window, storage):

    ij, window = ij_window
    band_path = new_scene[0].split('/')
    jpg_diff_local = f'/tmp/{band_path[7]}_{ij}_NDVI_DIFF.jpg'

    ndvi_local_f1, ndvi_mean_f1, co_jpg_f1, stats_f1 = calculate_ndvi(old_scene, ij_window, storage)
    ndvi_local_f2, ndvi_mean_f2, co_jpg_f2, stats_f2 = calculate_ndvi(new_scene, ij_window, storage)     
          
    ndvi_old = rasterio.open(ndvi_local_f1)
    ndvi_new = rasterio.open(ndvi_local_f2)

    profile = ndvi_old.profile
    profile.update(dtype='float64')
    profile.update(width=window.width)
    profile.update(height=window.height)

    no = ndvi_old.read(1).astype('float64')
    nn = ndvi_new.read(1).astype('float64')
    ndvi_cmp = ((nn - no) * (nn + no)).astype('float64')
    ndvi_cmp[0][0] = -1
    ndvi_cmp[0][1] = 1
    plt.imsave(jpg_diff_local, ndvi_cmp, cmap="RdYlGn")

    with open(jpg_diff_local, 'rb') as jpg_diff_file:
        co_jpg_diff = storage.put_cloudobject(jpg_diff_file, key=jpg_diff_local.replace('/tmp/', ''))

    return ij_window, co_jpg_f1, co_jpg_f2, co_jpg_diff, stats_f1, stats_f2

Using the selected parameters, get the identifiers of the selected tiles from Sentinel-2:

In [168]:
fexec = lithops.FunctionExecutor(
        backend='aws_lambda',
        storage='aws_s3',
        log_level='DEBUG',
        monitoring='rabbitmq',
        #max_workers=10,
        runtime='aws_lambda/lithops-ndvi-v39:01'  # Runtime for AWS Lambda
)

# Get data from all cells
iterdata = []
for scene_f1, scene_f2 in zip(scenes_f1, scenes_f2):
    for wd in windows:
        iterdata.append((scene_f1[0], scene_f2[0], wd))

# Execution
fexec.map(compute_ndvi_diff, iterdata)
results = fexec.get_result()
results

2022-06-10 14:33:20,450 [INFO] lithops.config -- Lithops v2.6.0
2022-06-10 14:33:20,452 [DEBUG] lithops.config -- Loading configuration from C:\Users\alega\PycharmProjects\geospatial-usecase\ndvi-diff\.lithops_config
2022-06-10 14:33:20,455 [DEBUG] lithops.config -- Loading Serverless backend module: aws_lambda
2022-06-10 14:33:20,456 [DEBUG] lithops.config -- Loading Storage backend module: aws_s3
2022-06-10 14:33:20,457 [DEBUG] lithops.storage.backends.aws_s3.aws_s3 -- Creating S3 client
2022-06-10 14:33:20,462 [INFO] lithops.storage.backends.aws_s3.aws_s3 -- S3 client created - Region: us-east-1
2022-06-10 14:33:20,463 [DEBUG] lithops.serverless.backends.aws_lambda.aws_lambda -- Creating AWS Lambda client
2022-06-10 14:33:20,464 [DEBUG] lithops.serverless.backends.aws_lambda.aws_lambda -- Creating Boto3 AWS Session and Lambda Client
2022-06-10 14:33:21,055 [INFO] lithops.serverless.backends.aws_lambda.aws_lambda -- AWS Lambda client created - Region: us-east-1
2022-06-10 14:33:21,05

2022-06-10 14:33:22,415 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00027 invoked (0.611s) - Activation ID: ea43d6b2-fbdf-4e10-af1d-707c1c17884b
2022-06-10 14:33:22,417 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00044 invoked (0.590s) - Activation ID: 91394049-5d2b-4bee-a60d-1a05ab35ee8c
2022-06-10 14:33:22,417 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00013 invoked (0.634s) - Activation ID: c6ab44c9-8e50-45f9-ac70-fae41c39e23b
2022-06-10 14:33:22,418 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00052 invoked (0.579s) - Activation ID: cf7c3288-be03-4b02-98eb-c7ea1c91cbf3
2022-06-10 14:33:22,421 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00036 invoked (0.604s) - Activation ID: 094657ae-9ad7-4c68-bcee-4d7f272e2cd9
2022-06-10 14:33:22,431 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00059 invoked (0.582s) - Activation ID: 0c2af065-aac4-470

2022-06-10 14:33:22,557 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00076 invoked (0.177s) - Activation ID: 747a702b-8770-4320-9287-fac29ae5e802
2022-06-10 14:33:22,567 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00069 invoked (0.224s) - Activation ID: cb1d672c-fd10-41ce-8a9c-d47b957d4d4b
2022-06-10 14:33:22,567 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00012 invoked (0.785s) - Activation ID: 33fd454e-4d67-4b78-9fd0-de20ca8d0e71
2022-06-10 14:33:22,568 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00081 invoked (0.163s) - Activation ID: 56b30eb2-57b1-471d-86f8-ddb40736dfa8
2022-06-10 14:33:22,569 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00084 invoked (0.145s) - Activation ID: 822f2fdd-cc46-4cc6-9d4c-3ae7fac157d5
2022-06-10 14:33:22,572 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00077 invoked (0.188s) - Activation ID: af616676-3fa4-40b

2022-06-10 14:33:22,781 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00139 invoked (0.116s) - Activation ID: a343a885-cf0d-481f-b74f-70032cac527b
2022-06-10 14:33:22,787 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00140 invoked (0.119s) - Activation ID: e7d502b6-7b20-4e28-91df-aa1512bf1faa
2022-06-10 14:33:22,796 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00137 invoked (0.138s) - Activation ID: c8ad180a-fe06-41eb-b49c-04209fc3d777
2022-06-10 14:33:22,799 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00143 invoked (0.120s) - Activation ID: 52deacd1-0213-4d2c-a90b-3813ae360c17
2022-06-10 14:33:22,801 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00141 invoked (0.128s) - Activation ID: 0ee9ae7f-3289-4cf3-a32c-dc33c6011532
2022-06-10 14:33:22,805 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00136 invoked (0.151s) - Activation ID: dfd7b522-54dc-4ab

2022-06-10 14:33:22,979 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00187 invoked (0.146s) - Activation ID: 617c80c1-2487-48c3-85e7-4cb543407302
2022-06-10 14:33:22,983 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00195 invoked (0.119s) - Activation ID: b0a68c8c-4fbe-4c6e-8624-a40e4997f44b
2022-06-10 14:33:22,991 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00196 invoked (0.122s) - Activation ID: d48b611f-226d-466a-8924-3e7203b98418
2022-06-10 14:33:22,991 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00189 invoked (0.151s) - Activation ID: f505aaf1-b2c9-4303-83c6-b3af1a5c7aff
2022-06-10 14:33:22,999 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00198 invoked (0.125s) - Activation ID: 9baef374-a8e4-4ae2-8027-a369b8b2d248
2022-06-10 14:33:23,002 [DEBUG] lithops.invokers -- ExecutorID 251169-2 | JobID M000 - Calls 00201 invoked (0.116s) - Activation ID: bc6d94d7-ebe5-427

...

2022-06-10 14:33:52,078 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got status from call 00139 - Activation ID: a343a885-cf0d-481f-b74f-70032cac527b - Time: 25.24 seconds
2022-06-10 14:33:52,079 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got status from call 00149 - Activation ID: 88422339-3ded-416f-83ad-c4df6bcf1d8a - Time: 25.36 seconds
2022-06-10 14:33:52,080 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00004 - Activation ID: 0b4047d4-4ce7-4ae1-af60-667dc308bee8
2022-06-10 14:33:52,084 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00009 - Activation ID: 19cfd74a-0410-43a5-a3bb-a21d412ad8b0
2022-06-10 14:33:52,086 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00013 - Activation ID: c6ab44c9-8e50-45f9-ac70-fae41c39e23b
2022-06-10 14:33:52,087 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00008 - Activation I

2022-06-10 14:33:52,251 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00086 - Activation ID: 786c6334-dc76-4c88-a76d-11f907919959
2022-06-10 14:33:52,254 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got status from call 00180 - Activation ID: 44cd89ac-0864-41a4-989b-ba03e9438e74 - Time: 25.46 seconds
2022-06-10 14:33:52,257 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00083 - Activation ID: b0d387b2-0b3c-4ebe-89ef-aacdb026dc9e
2022-06-10 14:33:52,257 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got status from call 00184 - Activation ID: 7652d3ec-fd2b-40a9-ade2-72af10ee65a4 - Time: 25.58 seconds
2022-06-10 14:33:52,265 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00093 - Activation ID: 9eace731-2efb-4cfd-ac3a-e63b1a4bda0e
2022-06-10 14:33:52,265 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got status from call 00187 - Activation I

2022-06-10 14:33:52,453 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00177 - Activation ID: 8f7ccd70-b9ba-4191-932d-48200a34508d
2022-06-10 14:33:52,455 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00178 - Activation ID: c2bcaf15-83b3-4696-87bf-e09df1001cf9
2022-06-10 14:33:52,456 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00179 - Activation ID: 91493606-41f9-4829-9530-d3b37cdcb989
2022-06-10 14:33:52,460 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00180 - Activation ID: 44cd89ac-0864-41a4-989b-ba03e9438e74
2022-06-10 14:33:52,468 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00184 - Activation ID: 7652d3ec-fd2b-40a9-ade2-72af10ee65a4
2022-06-10 14:33:52,480 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00191 - Activation ID: 8bfc5cb0-7904-4577-aa77-478c3332a3f0
2022

2022-06-10 14:33:52,788 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00024 - Activation ID: 4468cb44-6b20-49e4-a4df-a4cd20149519
2022-06-10 14:33:52,789 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00097 - Activation ID: e9729f12-e5ab-4421-998d-44c02017657b
2022-06-10 14:33:52,791 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00073 - Activation ID: ed6080bb-e6e3-4c31-bdcd-594846787256
2022-06-10 14:33:52,792 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got status from call 00211 - Activation ID: 76c634de-433a-40ec-bae4-47c7b00405bc - Time: 26.12 seconds
2022-06-10 14:33:52,793 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00145 - Activation ID: 119a9773-b85f-4617-ab9f-bdc03ddba142
2022-06-10 14:33:52,794 [DEBUG] lithops.future -- ExecutorID 251169-2 | JobID M000 - Got output from call 00047 - Activation ID: 7a1c8b46-a364-494c-

[(((0, 0), Window(col_off=0, row_off=0, width=1024, height=1024)),
  '',
  {'put': {'t0': 1654864421.4149456,
    't1': 1654864421.5983307,
    'bandwidth': 0.9892350393786573,
    'size': 181411.0},
   'get': {'t0': 1654864415.4554453,
    't1': 1654864418.2553203,
    'bandwidth': 0,
    'size': 0}},
  {'put': {'t0': 1654864426.114864,
    't1': 1654864426.1994588,
    'bandwidth': 1.6115661760461761,
    'size': 136330.0},
   'get': {'t0': 1654864421.627295,
    't1': 1654864423.2384555,
    'bandwidth': 0,
    'size': 0}}),
 (((0, 1), Window(col_off=1024, row_off=0, width=1024, height=1024)),
  '',
  {'put': {'t0': 1654864423.5094137,
    't1': 1654864423.7165363,
    'bandwidth': 1.173594971668,
    'size': 243078.0},
   'get': {'t0': 1654864416.549594,
    't1': 1654864419.3273113,
    'bandwidth': 0,
    'size': 0}},
  {'put': {'t0': 1654864428.8096569,
    't1': 1654864428.958949,
    'bandwidth': 1.5288270466146152,
    'size': 228242.0},
   'get': {'t0': 1654864423.7363756,
 

## 

## Get data statistics

In [169]:
# Generate plots
fexec.plot(dst='C:\\Users\\alega\\PycharmProjects\\geospatial-usecase\\ndvi-diff\\lithops' + str([lat, lon]))

2022-06-10 14:33:55,408 [INFO] lithops.executors -- ExecutorID 251169-2 - Creating execution plots


In [192]:
# Function to generate throughput statistics
def create_agg_bdwth_plot(res_write, res_read, dst):
    def compute_times_rates(start_time, d):
        x = np.array(d)
        tzero = start_time
        tr_start_time = x[:, 0] - tzero
        tr_end_time = x[:, 1] - tzero
        rate = x[:, 2]

        N = len(tr_start_time)
        runtime_rate_hist = np.zeros((N, len(runtime_bins)))

        for i in range(N):
            s = tr_start_time[i]
            e = tr_end_time[i]
            a, b = np.searchsorted(runtime_bins, [s, e])
            if b-a > 0:
                runtime_rate_hist[i, a:b] = rate[i]

        return {'start_time': tr_start_time,
                'end_time': tr_end_time,
                'rate': rate,
                'runtime_rate_hist': runtime_rate_hist}
    
    start_time = min((min(t['t0'] for t in res_write), (min(t['t0'] for t in res_read)))) - 1

    fig = pylab.figure(figsize=(8, 6))
    ax = fig.add_subplot(1, 1, 1)
    for datum, l in [(res_write, 'Aggregate Write Bandwidth'), (res_read, 'Aggregate Read Bandwidth')]:
        mb_rates = [(res['t0'], res['t1'], res['bandwidth']) for res in datum]
        max_seconds = int(max([mr[1]-start_time for mr in mb_rates])*1.2)
        max_seconds = 8 * round(max_seconds/8)
        runtime_bins = np.linspace(0, max_seconds, max_seconds)

        mb_rates_hist = compute_times_rates(start_time, mb_rates)

        ax.plot(mb_rates_hist['runtime_rate_hist'].sum(axis=0)/1000, label=l)

    ax.set_xlabel('Execution Time (sec)')
    ax.set_ylabel("GB/sec")
    ax.set_xlim(0, )
    ax.set_ylim(0, )
    pylab.legend()
    pylab.grid(True, axis='y')

    dst = os.path.expanduser(dst) if '~' in dst else dst

    fig.tight_layout()
    fig.savefig(dst, format='pdf')

### Prepare data

In [193]:
# Prepare read data size 
get_sz = 0
for scene in scenes_f1:
    obj = storage.head_object('sentinel-cogs', scene[0][0].replace('sentinel-cogs/', ''))
    obj2 = storage.head_object('sentinel-cogs', scene[0][1].replace('sentinel-cogs/', ''))
    get_sz = get_sz + float(obj.get('content-length')) + float(obj2.get('content-length'))

In [194]:
# Prepare throughput data
list_throughput = []
for worker in results:
    list_throughput.append(worker[4])
    list_throughput.append(worker[5])

read_results = [stat['get'] for stat in list_throughput]
write_results = [stat['put'] for stat in list_throughput]


# Calculate throughput write data
size_total_write = 0
for value in write_results:
    size_total_write = size_total_write + value.get('size')
    
# Calculate throughput read data
size_total_read = 0
for value in read_results:
    size_per_worker = get_sz / numberWorkers
    size_total_read = size_total_read + size_per_worker
    value['size'] = size_per_worker
    value['bandwidth'] = size_per_worker / (value.get('t1') - value.get('t0')) / 1e6

    
# Throughput read numeric:
throughput_interpolation_read = size_total_read / duration  # Bytes/second

# Throughput write numeric:
throughput_interpolation_write = size_total_write / duration  # Bytes/second

In [195]:
# Prepare duration data: 
startTime = set()
endTime = set()

for future in fexec.futures:
    for key in future.stats.keys():
        if key.endswith("worker_func_start_tstamp"):
            startTime.add(future.stats[key])
        if key.endswith("worker_end_tstamp"):
            endTime.add(future.stats[key])
            
            
duration = max(endTime) - min(startTime)


# Prepare number of workers:
numberWorkers=len(fexec.futures)

In [196]:
# Generate throughput graphic 
create_agg_bdwth_plot(read_results, write_results, fexec.executor_id + '-storage-kpi.pdf')

### Results

In [197]:
# Print results:
print(f"Data size bytes: {get_sz}")
mb = get_sz / 2**20
print(f"Data size megabytes: {mb}")
gb = mb / 1000
print(f"Data size gigabytes: {gb}")

print(f"Number of workers: {numberWorkers}")

print(f"Throughput write: {throughput_interpolation_write / 1024**2} MiB/s")
print(f"Throughput read: {throughput_interpolation_read / 1024**2} MiB/s")

print(f"Duration: {duration}")

Data size bytes: 845012747.0
Data size megabytes: 805.8669538497925
Data size gigabytes: 0.8058669538497925
Number of workers: 242
Throughput write: 4.869730361594683 MiB/s
Throughput read: 94.37855827450981 MiB/s
Duration: 17.07733130455017


## Get and plot the computed jpg diff tile image

In [ ]:
def get_jpg(data):
    file = '_'.join(data[0][1].key.split('_')[:5])
    
    if 'DIFF' in data[0][1].key:
        out_file = f'AwsData/{file}_NDVI_DIFF.jpg'
    else:
        out_file = f'AwsData/{file}_NDVI.jpg'
        
    jpgs = {}

    def get_window(data):
        ij_window, co_jpg = data
        row = ij_window[0][0]
        col = ij_window[0][1]
        jpg_stream = fexec.storage.get_cloudobject(co_jpg, stream=True)

        if row not in jpgs:
            jpgs[row] = [None]*11

        jpgs[row][col] = Image.open(jpg_stream)

    with ThreadPoolExecutor(max_workers=len(data)) as ex:
        fs = ex.map(get_window, data)

    # OJO CON EL SCENE_BAND PORQUE ESTO ESTA HECHO PARA 1 SOLA CELDA
    new_im = Image.new('RGB', (scene_band.width, scene_band.height))

    x_offset = 0
    y_offset = 0

    for row in sorted(jpgs.keys()):
        for im in jpgs[row]:
            new_im.paste(im, (x_offset, y_offset))
            x_offset += im.size[0]
        x_offset = 0
        y_offset += im.size[1]
        
    thumbnail_zise = (640, 640)
    new_im.thumbnail(thumbnail_zise)

    #fig = plt.figure(figsize=(10, 10))
    #plt.title(out_file)
    #plt.imshow(new_im)
    images[out_file] = new_im

In [ ]:
co_jpgs_f1 = [(res[0], res[1]) for res in results]
co_jpgs_f2 = [(res[0], res[2]) for res in results]
co_jpgs_diff = [(res[0], res[3]) for res in results]

images = {}
with ThreadPoolExecutor(max_workers=3) as ex:
    fs = ex.map(get_jpg, [co_jpgs_f1, co_jpgs_f2, co_jpgs_diff])

f, ax = plt.subplots(1,3, figsize=(18, 18))
i = 0
for j in sorted(images.keys()):
    ax[i].set_title(j)
    ax[i].imshow(images[j])
    i = i+1
plt.show() 

## Delete temporal files

In [187]:
keys = storage.list_keys(bucket=BUCKET_NAME)

In [188]:
for key in keys:
    storage.delete_object(bucket=BUCKET_NAME, key=key)